In [2]:
import pandas as pd
import numpy as np
import json
from os import listdir
from os.path import isfile, join

In [3]:
def load_files(filedir):
  with open(filedir) as f:
    file = json.loads(f.read())
    return file

# Pre-Processing

!git clone https://github.com/marcoramilli/MalwareTrainingSets/

In [4]:
mypath='/Users/fernanda/Documents/Hands On Machine Learning for Cybersecurity/Cybersecurity-MachineLearning/MalwareTrainingSets/trainingSets'
malwaretype=["APT1","Crypto","Locker","Zeus"]

apt1_files=[join(join(mypath,malwaretype[0]),f) for f in listdir(join(mypath,malwaretype[0])) if isfile(join(join(mypath,malwaretype[0]), f))]
crypto_files=[join(join(mypath,malwaretype[1]),f) for f in listdir(join(mypath,malwaretype[1])) if isfile(join(join(mypath,malwaretype[1]), f))]
locker_files=[join(join(mypath,malwaretype[2]),f) for f in listdir(join(mypath,malwaretype[2])) if isfile(join(join(mypath,malwaretype[2]), f))]
zeus_files=[join(join(mypath,malwaretype[3]),f) for f in listdir(join(mypath,malwaretype[3])) if isfile(join(join(mypath,malwaretype[3]), f))]

apt1_files = [load_files(filedir=name) for name in apt1_files]
crypto_files = [load_files(filedir=name) for name in crypto_files]
locker_files = [load_files(filedir=name) for name in locker_files]
zeus_files = [load_files(filedir=name) for name in zeus_files]

In [5]:
# archive structure:
print(apt1_files[0].keys())
print(crypto_files[0].keys())
print(locker_files[0].keys())
print(zeus_files[0].keys())
print("\n")
print('properties apt1 file 0:',len(apt1_files[0]['properties'].keys()))
print('properties crypto file 0:',len(crypto_files[0]['properties'].keys()))
print('properties locker file 0:',len(locker_files[0]['properties'].keys()))
print('properties zeus file 0:',len(zeus_files[0]['properties'].keys()))
#each kind of malware has different properties in its files

dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])


properties apt1 file 0: 22
properties crypto file 0: 18
properties locker file 0: 17
properties zeus file 0: 34


In [6]:
apt1_prop = []
crypto_prop = []
locker_prop = []
zeus_prop = []

for file in apt1_files:
  for prop in file['properties']:
    if (prop not in apt1_prop):
        apt1_prop.append(prop)
for file in crypto_files:
  for prop in file['properties']:
    if (prop not in crypto_prop):
        crypto_prop.append(prop)
for file in locker_files:
  for prop in file['properties']:
    if (prop not in locker_prop):
        locker_prop.append(prop)
for file in zeus_files:
  for prop in file['properties']:
    if (prop not in zeus_prop):
        zeus_prop.append(prop)

#check common properties
props = []
for prop in apt1_prop:
  if (prop in crypto_prop and prop in locker_prop and zeus_prop):
    props.append(prop)

print('common properties:',len(props))

common properties: 52


In [25]:
## create dataframe
df=pd.DataFrame(columns=props)
        
hexa=apt1_files[0]['properties'].get(props[0])

def append_df(file,properties):
    line=[]
    for i in range(0,len(properties)):
        try: line.append(file['properties'].get(props[i]))
        except: line.append()
    return line

for f in apt1_files:
    df.loc[len(df)]=append_df(f,props)
for f in crypto_files:
    df.loc[len(df)]=append_df(f,props)
for f in locker_files:
    df.loc[len(df)]=append_df(f,props)
for f in zeus_files:
    df.loc[len(df)]=append_df(f,props)

In [27]:
df.describe()

,file_access,pe_imports,reg_read,sig_antisandbox_sleep,reg_write,pe_sec_character,label,api_resolv,pe_sec_entropy,pe_sec_name,...,sig_packer_entropy,sig_exec_crash,sig_stealth_network,sig_network_cnc_http,sig_origin_langid,sig_antiav_servicestop,sig_process_interest,sig_modifies_certs,sig_encrypted_ioc,sig_injection_createremotethread
count,3696,4557,3511,263,1529,4750,4762,4028,4750,4750,...,2934,6,70,42,68,18,216,47,173,308
unique,1908,1212,861,209,501,200,4,1321,1850,351,...,831,1,1,22,1,7,54,1,71,160
top,bda9f42e 3c6b9e80 06416233,69fe6141 9346f95b cec36c4f 37aa7177 dfd10ba1 6...,1fe39017 df16f4d5,9f4c2c60,75ff7b0e a72671f7,d78064fc eb95869a ad8225f3,Crypto,e6e88ee5 8c70d8b3 090a162f 2572f7e8 4b03b39a a...,63790b7d 566d705e 247ea3d9 6915b647,c57b5e67 46040620 1f9f1073,...,5617b9b7,,,bb03f7af 543b370c b26cf15b,,10fc4023 e788ca34 e557c1c1 cc411e6c 7a7e8242,cde09bcd,,23037168,69eb7452
freq,96,106,1140,4,182,1089,2020,240,78,1114,...,78,6,70,6,68,6,28,47,21,2


In [70]:
#drop columns with more than 40% of values=null
dfsomanulo=df.isna().sum().sort_values(ascending=False)
l,c=df.shape
drp=[]
for i in range(0,len(dfsomanulo)):
    if dfsomanulo[i]/l>0.4:
        drp.append(dfsomanulo.index[i])
df.drop(drp,axis=1)

,file_access,pe_imports,reg_read,pe_sec_character,label,api_resolv,pe_sec_entropy,pe_sec_name,sig_antivirus_virustotal,file_read,reg_access,str,sig_injection_rwx,sig_packer_entropy
0,d41d8cd9 913f9c49 96da6d37,e6e91a7b daa85be1 9f22226d 21b6aebe 76e954be e...,c1c40440 0aaa8742 2e5d8aa3 ab9dfd08 d2ef061c 6...,9271b28c 2f0acaf7 ad8225f3 2f0acaf7,APT1,2435d74b 8e16a687 cc9bfd1e 1b64cca1 bd79f8c0 9...,e4f25f73 e6d2fa08 89f4d802 32a522c7,916e7571 e8997399 b7b66a05 1f9f1073,bd38e569 92c012f4 befdf1a9 0004a5f9 65357e06 8...,d41d8cd9 913f9c49 96da6d37,c576a841 c1c40440 c1c40440 0aaa8742 2e5d8aa3 8...,916e7571 ec73657d e4787bb1 1f9f1073 b8f4bfae b...,None,None
1,d41d8cd9 913f9c49 96da6d37 3d801aa5 93b002fa 9...,bfe5f955 8c00ba37 95dfa11f a2d78c4b 888b1f30 b...,fb216d9e cbb84746 1471e4e0 c4ca4238 35e4625d 0...,9271b28c 2f0acaf7 ad8225f3 2f0acaf7 2f0acaf7,APT1,b55144b0 a906dfcd 1a8c9e76 192010fe 5cba9199 0...,675b17a7 4b71b781 4bcb5644 94e0b709 02d63d7f,916e7571 e8997399 b7b66a05 af5cc73b 1f9f1073,a07a88da a8046920 dfc8bd31 f7e5ca1f f7e5ca1f e...,d41d8cd9 913f9c49 96da6d37 3d801aa5 93b002fa 9...,c0f87020 fb216d9e a13a40f5 10f71998 8512ae7d 1...,916e7571 ec73657d e4787bb1 dc02c617 d824eeec e...,,None
2,3d801aa5 245c2557 98e83379 2e5d8aa3 245c2557 9...,a409f4dc f0e90580 c9fab33e 95d1df62 301c6841 3...,9ea6efd4 40d70447 7e523f39 40d70447 fb216d9e c...,9271b28c 2f0acaf7 ad8225f3,APT1,ec2b83df 02eb14e7 d847e1f5 b5d56eb9 87b97cb4 4...,e4f25f73 9da6b4c8 282bc12f,916e7571 e8997399 b7b66a05,33966d5b af573f20 3eeeeb4a 97bb6856 b382891a a...,3d801aa5 245c2557 98e83379 8b4939bc 080a35bc c...,c0f87020 9ea6efd4 9ea6efd4 40d70447 c0f87020 7...,916e7571 ec73657d e4787bb1 47ed7c0c 47ed7c0c 4...,None,None
3,d41d8cd9 913f9c49 96da6d37,c9fab33e f1210d90 bf6b5066 404da61d 821b7dbb 8...,2e5d8aa3 ab9dfd08 d2ef061c 6f8d1b93,9271b28c 2f0acaf7 ad8225f3,APT1,8e16a687 cc9bfd1e 1b64cca1 bd79f8c0 06336c86 4...,55b28296 5c2a5110 bb432ad8,916e7571 e8997399 b7b66a05,None,d41d8cd9 913f9c49 96da6d37,efc50fc9 a53108f7 a53108f7 7fb47ee6 2e5d8aa3 8...,916e7571 ec73657d e4787bb1 9349df31 05d559f2 4...,None,None
4,3d801aa5 547d84b8 98e83379 ab3049da 2d2ac038 4...,c39603a0 f659f996 21a39751 dc11a7db a409f4dc a...,9ea6efd4 40d70447 7e523f39 40d70447,9271b28c 2f0acaf7 ad8225f3 2f0acaf7,APT1,ec2b83df 02eb14e7,cd69931f b5476352 d284a497 001aa5ac,916e7571 e8997399 b7b66a05 1f9f1073,None,8b4939bc 080a35bc c0f87020,c0f87020 9ea6efd4 9ea6efd4 40d70447 c0f87020 7...,916e7571 ec73657d e4787bb1 1f9f1073 1c351c50 8...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757,None,839ca61c 82b1a48c a5c1f7de cec36c4f 805da537 9...,None,d78064fc eb95869a 1bba146b,Zeus,1b0321cf 4117a7f6 46a9fc21 f195feb6 5e339354 a...,f7ddd489 521a2bfc fb68b21e,c57b5e67 46040620 1f9f1073,7e579992 db894587 1d27c4f3 ead1c7a4 db894587 9...,None,None,1f9f1073 e473645b c688578b 4db6dd06 6d092e35 1...,,7170d70b
4758,3d801aa5 03ee74ec 06416233 bda9f42e 03ee74ec 0...,2b591dfa 3d397952 17f93d0f 6c335e1e 3690fd46 0...,1fe39017 df16f4d5 0f4137ed ec4d1eb3 e33cfe8d 6...,9271b28c 2f0acaf7 ad8225f3 2f0acaf7,Zeus,a906dfcd f60a30d8 8c70d8b3 38c96c1d 70d48670 4...,917c9224 73f148b7 f5146520 00d034f2,916e7571 e8997399 b7b66a05 1f9f1073,17c7c628 83bdbaeb fefbf6ae 9bc62134 9bc62134 b...,bda9f42e 2af72f10 06416233 7b946538 0f7b7f07 0...,efc50fc9 1fe39017 1fe39017 df16f4d5 efc50fc9 a...,916e7571 ec73657d e4787bb1 1f9f1073 3d0c9f8d 5...,,None
4759,None,69fe6141 9346f95b cec36c4f 37aa7177 dfd10ba1 6...,None,9271b28c 2f0acaf7 ad8225f3 2f0acaf7,Zeus,e6e88ee5 8c70d8b3 090a162f 2572f7e8 4b03b39a a...,3259433f a81596e6 a0b6088d c8879c5f,916e7571 e8997399 b7b66a05 1f9f1073,197b1a45 4ac06cbb 245cb7dc ab825f73 4ac06cbb f...,None,e0bda819 8eb58dd5 8eb58dd5 e6057ba0,916e7571 ec73657d e4787bb1 1f9f1073 ad433c92 3...,,24d210b1
4760,3d801aa5 47a66599 2deeeeda 47a66599 2deeeeda 0...,839ca61c 82b1a48c a5c1f7de cec36c4f 805da537 9...,9ea6efd4 40d70447 7e523f39 40d70447 c1c40440 0...,d78064fc eb95869a ad8225f3,Zeus,010c5f8b 79c97b83 2